# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [4]:
db = client["ironhack"]
c = db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [10]:
filter_ = {"name":"Babelgum"}
project_ = {"_id":0,"name":1}
name_Babelgum = list(c.find(filter_, project_))
name_Babelgum

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [16]:
filter_ = {"number_of_employees": {"$gt":5000}}
more_5000_employees= list(c.find(filter_).sort("number_of_employees", 1).limit(20))
more_5000_employees


[{'_id': ObjectId('52cdef7c4bab8bd675297e49'),
  'name': 'Nintendo',
  'permalink': 'nintendo',
  'crunchbase_url': 'http://www.crunchbase.com/company/nintendo',
  'homepage_url': 'http://nintendo.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'NintendoAmerica',
  'category_code': 'games_video',
  'number_of_employees': 5080,
  'founded_year': 1889,
  'founded_month': 9,
  'founded_day': 23,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'games, console',
  'alias_list': '',
  'email_address': 'noalegal@noa.nintendo.com',
  'phone_number': '1-800-255-3700 ',
  'description': '',
  'created_at': 'Sat Dec 22 13:27:00 UTC 2007',
  'updated_at': 'Thu Sep 05 12:41:16 UTC 2013',
  'overview': '<p>Nintendo, a technology company widely known for its line of game consoles, was actually founded in 1889 by Fusajiro Yamauchi. It began as a card game company and evolved into one of the largest Japanese c

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [91]:
filter_1 = {"founded_year":{"$gte":2000}}
filter_2 = {"founded_year":{"$lte":2005}}
all_filters = [filter_1, filter_2]

project_ = {"_id":0, "name":1, "founded_year":1}

founded_2000_2005 = list(c.find({"$and": all_filters}, project_))

founded_2000_2005[:5]

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [43]:
filter_1 = {"ipo.valuation_amount": {"$gt": 100000000}}
filter_2 = {"founded_year": {"$lt": 2010}}
all_filters = [filter_1, filter_2]

project_ = {"_id": 0, "name": 1, "ipo": 1}

big_valuation = list(c.find({"$and": all_filters}, project_))

big_valuation[:2]

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [56]:
filter_1 = {"number_of_employees": {"$lt": 1000}}
filter_2 = {"founded_year": {"$lt": 2005}}
all_filters = [filter_1, filter_2]

ex_5 = list(c.find({"$and": all_filters}).sort("number_of_employees", 1).limit(10))

ex_5[:2]

[{'_id': ObjectId('52cdef7c4bab8bd675297dbc'),
  'name': 'Skype',
  'permalink': 'skype',
  'crunchbase_url': 'http://www.crunchbase.com/company/skype',
  'homepage_url': 'http://www.skype.com',
  'blog_url': 'http://blogs.skype.com',
  'blog_feed_url': 'http://feeds.feedburner.com/shareskypeen',
  'twitter_username': 'skype',
  'category_code': 'software',
  'number_of_employees': 0,
  'founded_year': 2003,
  'founded_month': 8,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': '',
  'tag_list': 'phone, voip, sweden, skype',
  'alias_list': '',
  'email_address': '',
  'phone_number': '94117733',
  'description': 'VoIP Software ',
  'created_at': 'Tue Jul 03 05:20:31 UTC 2007',
  'updated_at': 'Wed Dec 11 13:03:57 UTC 2013',
  'overview': '<p>Skype is a software application that allows users to make voice and video calls and chats over the Internet. Calls to other users within the Skype service are free, while c

### 6. All the companies that don't include the `partners` field.

In [49]:
filter_1 = {"partners": {"$exists":False}}

list(c.find(filter_1))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [53]:
filter_1 = {"category_code": {"$type": "null"}}

list(c.find(filter_1))[:2]

[{'_id': ObjectId('52cdef7c4bab8bd6752980f6'),
  'name': 'Collective',
  'permalink': 'collective',
  'crunchbase_url': 'http://www.crunchbase.com/company/collective',
  'homepage_url': None,
  'blog_url': None,
  'blog_feed_url': None,
  'twitter_username': None,
  'category_code': None,
  'number_of_employees': None,
  'founded_year': None,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': None,
  'alias_list': None,
  'email_address': None,
  'phone_number': None,
  'description': None,
  'created_at': 'Thu Sep 26 13:15:02 UTC 2013',
  'updated_at': 'Thu Sep 26 13:15:02 UTC 2013',
  'overview': None,
  'image': None,
  'products': [],
  'relationships': [],
  'competitions': [],
  'providerships': [],
  'total_money_raised': '$0',
  'funding_rounds': [],
  'investments': [],
  'acquisition': None,
  'acquisitions': [],
  'offices': [],
  'milestones': [],
  'ipo': N

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [59]:
filter_1 = {"number_of_employees": {"$gte": 100}}
filter_2 = {"number_of_employees": {"$lt": 1000}}
all_filters = [filter_1, filter_2]

project = {"_id":0, "name": 1, "number_of_employees": 1}

list(c.find({"$and": all_filters}, project))[:5]

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [65]:
list(c.find().sort("ipo.valuation_amount", -1))[:2]

[{'_id': ObjectId('52cdef7e4bab8bd67529a8b4'),
  'name': 'GREE',
  'permalink': 'gree',
  'crunchbase_url': 'http://www.crunchbase.com/company/gree',
  'homepage_url': 'http://www.gree-corp.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'gree_corp',
  'category_code': 'games_video',
  'number_of_employees': 700,
  'founded_year': 2004,
  'founded_month': 12,
  'founded_day': 7,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'mobile-web, japan, tokyo, social-network, mobile-social-network, mobile-games',
  'alias_list': None,
  'email_address': 'inquiry@gree-corp.com',
  'phone_number': '',
  'description': 'Internet media business,SNS,  free game',
  'created_at': 'Sat Dec 20 16:42:57 UTC 2008',
  'updated_at': 'Tue Jan 01 21:37:04 UTC 2013',
  'overview': '<p>GREE provides Japan&#8217;s leading mobile social network, and is at the forefront of mobile technology. GREE was ranked as Japan&#82

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [69]:
list(c.find().sort("number_of_employees", 1).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297d92'),
  'name': 'Flektor',
  'permalink': 'flektor',
  'crunchbase_url': 'http://www.crunchbase.com/company/flektor',
  'homepage_url': 'http://www.flektor.com',
  'blog_url': 'http://www.flektor-blog.com',
  'blog_feed_url': 'http://www.flektor-blog.com/video_editing_software/index.rdf',
  'twitter_username': None,
  'category_code': 'games_video',
  'number_of_employees': None,
  'founded_year': None,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'flektor, photo, video',
  'alias_list': None,
  'email_address': None,
  'phone_number': None,
  'description': None,
  'created_at': 'Thu May 31 21:11:51 UTC 2007',
  'updated_at': 'Sat Nov 05 08:42:23 UTC 2011',
  'overview': '<p>Flektor is a rich-media mash-up platform that enables consumers to create, remix and share photos and videos on the internet without the need for adva

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [70]:
filter_1 = {"founded_month": {"$gt": 6}}

list(c.find(filter_1).limit(1000))[:2]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'permalink': 'abc2',
  'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
  'homepage_url': 'http://wetpaint-inc.com',
  'blog_url': 'http://digitalquarters.net/',
  'blog_feed_url': 'http://digitalquarters.net/feed/',
  'twitter_username': 'BachelrWetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'founded_year': 2005,
  'founded_month': 10,
  'founded_day': 17,
  'deadpooled_year': 1,
  'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
  'alias_list': '',
  'email_address': 'info@wetpaint.com',
  'phone_number': '206.859.6300',
  'description': 'Technology Platform Company',
  'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
  'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
  'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and mon

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [73]:
filter_1 = {"founded_year": {"$lt": 2000}}
filter_2 = {"acquisition.price_amount": {"$gt": 10000000}}
all_filters = [filter_1, filter_2]

list(c.find({"$and": all_filters}))[:2]

[{'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'name': 'Postini',
  'permalink': 'postini',
  'crunchbase_url': 'http://www.crunchbase.com/company/postini',
  'homepage_url': 'http://postini.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': None,
  'category_code': 'web',
  'number_of_employees': None,
  'founded_year': 1999,
  'founded_month': 6,
  'founded_day': 2,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': '',
  'alias_list': None,
  'email_address': '',
  'phone_number': '888.584.3150',
  'description': None,
  'created_at': 'Fri Jun 08 12:19:51 UTC 2007',
  'updated_at': 'Sat Aug 13 18:02:34 UTC 2011',
  'overview': '<p>Postini focuses on two main issues: security and compliance. Postini states that it handles more than 1 billion messages everyday and protects more than 35,000 businesses worldwide.</p>\n\n<p>Postini offers solutions that protect your company from malicious internet a

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [75]:
filter_1 = {"acquisition.acquired_year": {"$lt": 2010}}

project = {"_id":0, "name": 1, "acquisition": 1}

list(c.find(filter_1, project).sort("acquisition.price_amount", 1))[:2]

[{'name': 'Gizmoz',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.gizmoz.com',
   'source_description': 'TechCrunch',
   'acquired_year': 2009,
   'acquired_month': 12,
   'acquired_day': 15,
   'acquiring_company': {'name': 'Daz 3d', 'permalink': 'daz-3d'}}},
 {'name': 'Joost',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.techcrunch.com/2009/11/24/joost-acquired-adconion/',
   'source_description': 'Joost Is Now Officially Dead â€“ Assets Acquired By Adconion Media Group',
   'acquired_year': 2009,
   'acquired_month': 11,
   'acquired_day': 24,
   'acquiring_company': {'name': 'Adconion Media Group',
    'permalink': 'adconion-media-group'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [84]:
filter_1 = {"founded_year": {"$ne": None}}
project = {"_id": 0, "name": 1, "founded_year": 1}

list(c.find(filter_1, project).sort("founded_year", 1))

[{'name': 'Alstrasoft', 'founded_year': 1800},
 {'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'US Army', 'founded_year': 1800},
 {'name': 'DuPont', 'founded_year': 1802},
 {'name': 'McKesson', 'founded_year': 1833},
 {'name': 'Bachmann Industries', 'founded_year': 1833},
 {'name': 'Bertelsmann', 'founded_year': 1835},
 {'name': 'Accuity', 'founded_year': 1836},
 {'name': 'CENTRA', 'founded_year': 1839},
 {'name': 'WeGame', 'founded_year': 1840},
 {'name': 'VideoSurf', 'founded_year': 1840},
 {'name': 'VideoSurf', 'founded_year': 1840},
 {'name': 'The Economist Group', 'founded_year': 1843},
 {'name': 'Pearson', 'founded_year': 1844},
 {'name': 'Associated Press', 'founded_year': 1846},
 {'name': 'Beloit College', 'founded_year': 1846},
 {'name': 'Siemens', 'founded_year': 1847},
 {'name': 'Stiefel', 'founded_year': 1847},
 {'name': 'Pfizer', 'founded_year': 1848},
 {'name': 'Nykredit Realkredit', 'founded_year': 1851},
 {'name': 'Corning', 'founded_year': 1851},
 {'name': 'L

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [86]:
filter_1 = {"founded_month": {"$lte": 7}}

list(c.find(filter_1).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'permalink': 'facebook',
  'crunchbase_url': 'http://www.crunchbase.com/company/facebook',
  'homepage_url': 'http://facebook.com',
  'blog_url': 'http://blog.facebook.com',
  'blog_feed_url': 'http://blog.facebook.com/atom.php',
  'twitter_username': 'facebook',
  'category_code': 'social',
  'number_of_employees': 5299,
  'founded_year': 2004,
  'founded_month': 2,
  'founded_day': 1,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': '',
  'tag_list': 'facebook, college, students, profiles, network, online-communities, social-networking',
  'alias_list': '',
  'email_address': '',
  'phone_number': '',
  'description': 'Social network',
  'created_at': 'Fri May 25 21:22:15 UTC 2007',
  'updated_at': 'Thu Nov 21 19:40:55 UTC 2013',
  'overview': '<p>Facebook is the world&#8217;s largest social network, with over <a href="http://techcrunch.com/2013/07/24/facebook-gr

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [92]:
filter_1 = {"category_code": "web"}
filter_2 = {"number_of_employees": {"$gt": 4000}}
all_filters = [filter_1, filter_2]

list(c.find({"$and": all_filters}).sort("number_of_employees", 1))[:2]

[{'_id': ObjectId('52cdef7c4bab8bd67529822a'),
  'name': 'Expedia',
  'permalink': 'expedia',
  'crunchbase_url': 'http://www.crunchbase.com/company/expedia',
  'homepage_url': 'http://www.expedia.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'Expedia',
  'category_code': 'web',
  'number_of_employees': 4400,
  'founded_year': 1996,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'travel, flights, hotels, tickets, airline, vacation, travel-comparison, travel-search',
  'alias_list': '',
  'email_address': '',
  'phone_number': '1-800-EXPEDIA',
  'description': '',
  'created_at': 'Sun Feb 10 03:29:12 UTC 2008',
  'updated_at': 'Sun Dec 08 07:03:33 UTC 2013',
  'overview': '<p>Expedia.com is an Internet-based travel agency and a part of Expedia, Inc.. It books airline tickets, hotel reservations, car rentals, cruises, vacation packages, and vari

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [93]:
filter_1 = {"acquisition.price_currency_code": "EUR"}
filter_2 = {"acquisition.price_amount": {"$gt": 10000000}}
all_filters = [filter_1, filter_2]

list(c.find({"$and": all_filters}))[:2]

[{'_id': ObjectId('52cdef7c4bab8bd675297f02'),
  'name': 'ZYB',
  'permalink': 'zyb',
  'crunchbase_url': 'http://www.crunchbase.com/company/zyb',
  'homepage_url': 'http://zyb.com',
  'blog_url': 'http://blog.zyb.com/',
  'blog_feed_url': 'http://blog.zyb.com/index.php/feed/',
  'twitter_username': None,
  'category_code': 'mobile',
  'number_of_employees': 25,
  'founded_year': None,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'mobile, social-networking, socialnetwork',
  'alias_list': '',
  'email_address': 'contact@zyb.com',
  'phone_number': '',
  'description': 'Mobile Social Networking',
  'created_at': 'Wed Aug 22 05:14:51 UTC 2007',
  'updated_at': 'Tue Jul 02 03:00:50 UTC 2013',
  'overview': '<p>ZYB is a mobile phone utility and social networking site. ZYB started in 2005 as a european service to provide easy back-up services for mobile phones. Using t

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [95]:
filter_1 = {"acquisition.acquired_month": {"$lte": 3}}

project = {"_id": 0, "name": 1, "acquisition": 1}

list(c.find(filter_1, project))[:2]

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [98]:
filter_1 = {"founded_year": {"$gt": 2000, "$lt": 2010}}
filter_2 = {"acquisition.acquired_year": {"$gte": 2011}}
filter_3 = {"acquisition.acquired_year": {"$type": "null"}}

list(c.find({"$and": [filter_1, {"$or": [filter_2, filter_3]}]}))[:2]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'permalink': 'abc2',
  'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
  'homepage_url': 'http://wetpaint-inc.com',
  'blog_url': 'http://digitalquarters.net/',
  'blog_feed_url': 'http://digitalquarters.net/feed/',
  'twitter_username': 'BachelrWetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'founded_year': 2005,
  'founded_month': 10,
  'founded_day': 17,
  'deadpooled_year': 1,
  'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
  'alias_list': '',
  'email_address': 'info@wetpaint.com',
  'phone_number': '206.859.6300',
  'description': 'Technology Platform Company',
  'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
  'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
  'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and mon

### 20. All the companies that have been 'deadpooled' after the third year.

In [101]:
filter_1 = {"deadpooled_year": {"$gt": 3}}

list(c.find(filter_1))[:2]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'permalink': 'omnidrive',
  'crunchbase_url': 'http://www.crunchbase.com/company/omnidrive',
  'homepage_url': 'http://www.omnidrive.com',
  'blog_url': 'http://www.omnidrive.com/blog',
  'blog_feed_url': 'http://feeds.feedburner.com/omnidrive',
  'twitter_username': 'Nomadesk',
  'category_code': 'network_hosting',
  'number_of_employees': None,
  'founded_year': 2005,
  'founded_month': 11,
  'founded_day': 1,
  'deadpooled_year': 2008,
  'deadpooled_month': 9,
  'deadpooled_day': 15,
  'deadpooled_url': '',
  'tag_list': 'storage, sharing, edit, online',
  'alias_list': None,
  'email_address': 'info@omnidrive.com',
  'phone_number': '660-675-5052',
  'description': None,
  'created_at': 'Sun May 27 03:25:32 UTC 2007',
  'updated_at': 'Tue Jul 02 22:48:04 UTC 2013',
  'overview': '<p>Currently in public beta, Omnidrive makes it easy to access, edit and share your files from any computer with a web browser. The s